# Bases de connaissance : interroger Wikidata à l'aide de requêtes SPARQL

## Imports

In [2]:
from datetime import datetime as dt
from SPARQLWrapper import SPARQLWrapper, JSON

## Obtenir la liste des politiciens belges

In [3]:
# Retrieve results from SPARQL
endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
sparql = SPARQLWrapper(endpoint)

# P27 = country of citizenship 
# Q31 = Belgium
# P106 = Opccupation
# Q82955 = Politician
# P569 = date of birth
# P570 = date of death
statement = """
SELECT DISTINCT ?article ?articleLabel WHERE {
    SERVICE wikibase:label { bd:serviceParam wikibase:language "fr" . }
{
    SELECT DISTINCT ?article WHERE {
 ?article wdt:P27 wd:Q31.

}
   }
    
}
ORDER BY ?articleLabel
"""

sparql.setQuery(statement)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

rows = results['results']['bindings']
print(f"\n{len(rows)} Belgian articles written in french\n")
print(rows[:10])

## Filtrer pour n'afficher que les noms contenant "Alphonse"

In [3]:
name_filter = 'Alphonse'
max_results = 10

date_format = "%Y-%m-%dT%H:%M:%SZ"
filtered_rows = [row for row in rows if name_filter in row['personLabel']['value']]
print(f"Displaying the first {max_results}:\n")
for row in filtered_rows[:max_results]:
    try:
        birth_date = dt.strptime(row['dateBirth']['value'], date_format)
        birth_year = birth_date.year
    except ValueError:
        birth_year = "????"
    try:
        death_date = dt.strptime(row['dateDeath']['value'], date_format)
        death_year = death_date.year
    except ValueError: # unknown death date
        death_year = "????"
    except KeyError: # still alive
        death_year = ""
    print(f"{row['personLabel']['value']} ({birth_year}-{death_year})")

Displaying the first 10:

Adolphe-Alphonse Goupy de Beauvolers (1796-1869)
Alphonse Bonenfant (1908-1983)
Alphonse Bosch (1886-1944)
Alphonse Brenez (1862-1933)
Alphonse Briart (1864-1936)
Alphonse Carpentier (1869-1934)
Alphonse Casse (1846-1923)
Alphonse Clignez (1886-1965)
Alphonse Collet (1905-1971)
Alphonse De Haene (1863-1939)


## Pour en savoir plus

- [Le projet Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page)
- [Aide à la construction de requêtes](https://query.wikidata.org/)